## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chokurdakh,RU,70.6333,147.9167,-13.85,98,79,8.37,broken clouds
1,1,Saldanha,ZA,-33.0117,17.9442,63.68,90,0,13.27,clear sky
2,2,Khatanga,RU,71.9667,102.5000,-27.47,100,49,2.80,scattered clouds
3,3,Hermanus,ZA,-34.4187,19.2345,66.90,78,80,18.34,broken clouds
4,4,Port Hedland,AU,-20.3167,118.5667,98.65,39,16,17.27,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Geraldton,AU,-28.7667,114.6000,86.41,58,0,11.50,clear sky
20,20,Ransang,PH,8.9603,117.5828,84.31,67,94,4.09,overcast clouds
26,26,Saint-Philippe,RE,-21.3585,55.7679,80.15,84,37,10.71,scattered clouds
35,35,Vaini,TO,-21.2000,-175.2000,80.28,71,88,16.15,overcast clouds
37,37,Inhambane,MZ,-23.8650,35.3833,75.09,82,0,7.45,clear sky
40,40,Rikitea,PF,-23.1203,-134.9692,78.93,71,2,14.38,clear sky
43,43,Georgetown,MY,5.4112,100.3354,86.90,71,20,4.61,few clouds
46,46,Atuona,PF,-9.8000,-139.0333,77.32,80,20,13.96,light rain
48,48,Alofi,NU,-19.0595,-169.9187,81.09,74,99,13.35,light rain
49,49,Avarua,CK,-21.2078,-159.7750,75.25,88,75,8.05,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                151
City                   151
Country                151
Lat                    151
Lng                    151
Max Temp               151
Humidity               151
Cloudiness             151
Wind Speed             151
Current Description    151
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                151
City                   151
Country                151
Lat                    151
Lng                    151
Max Temp               151
Humidity               151
Cloudiness             151
Wind Speed             151
Current Description    151
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Geraldton,AU,86.41,clear sky,-28.7667,114.6000,
20,Ransang,PH,84.31,overcast clouds,8.9603,117.5828,
26,Saint-Philippe,RE,80.15,scattered clouds,-21.3585,55.7679,
35,Vaini,TO,80.28,overcast clouds,-21.2000,-175.2000,
37,Inhambane,MZ,75.09,clear sky,-23.8650,35.3833,
40,Rikitea,PF,78.93,clear sky,-23.1203,-134.9692,
43,Georgetown,MY,86.90,few clouds,5.4112,100.3354,
46,Atuona,PF,77.32,light rain,-9.8000,-139.0333,
48,Alofi,NU,81.09,light rain,-19.0595,-169.9187,
49,Avarua,CK,75.25,light rain,-21.2078,-159.7750,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.count()

City                   146
Country                146
Max Temp               146
Current Description    146
Lat                    146
Lng                    146
Hotel Name             146
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd> and <dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))